`Unsharp Masking`
---

---

In [2]:
import numpy as np

def convolve(image: np.ndarray, kernel: np.ndarray) -> np.ndarray:
    # boundaries for kernel window
    kernel_x: int
    kernel_y: int
    kernel_y, kernel_x = kernel.shape

    # range for loops
    image_y: int
    image_x: int
    image_y, image_x = image.shape

    # dynamic pad sizing based on kernel dimensions
    pad_y: int
    pad_x: int
    pad_y, pad_x = kernel_y // 2, kernel_x // 2 


    filtered_image: np.ndarray # init output matrix
    paddded_image: np.ndarray
    window: np.ndarray # sliding frame
    filtered_image = np.zeros_like(a = image, dtype = np.float32)                                         
    paddded_image = np.pad(array = image, pad_width = (( pad_y, pad_x ), ( pad_x, pad_y )), mode = 'constant', constant_values = 0)

    # Convolve
    for y in range(image_y):
        for x in range(image_x):
            # Sliding window
            window = paddded_image[ y : y + kernel_y, x : x + kernel_x ]
            # Apply mask
            filtered_image[ y, x ] = np.sum(window * kernel)

    return filtered_image


def unsharp_mask(image: np.ndarray, A: int | float = 1):

    unsharp_kernel: np.ndarray = np.array([[0, -1, 0],
                                           [-1, (A + 4), -1],
                                           [0, -1, 0]], dtype=np.float32)
    unsharp_kernel /= A

    unsharp_image: np.ndarray = convolve(image, unsharp_kernel)
    return (unsharp_image := np.clip(unsharp_image, 0, 255))


matrix = np.array([[10, 11, 9, 25, 22],
                   [8, 10, 9, 26, 28],
                   [9, 8, 9, 24, 25],
                   [11, 11, 12, 23, 22],
                   [10, 11, 9, 22, 25]], dtype = np.float32)


print(unsharp_mask(matrix, A = 1))



[[31. 26.  0. 68. 57.]
 [11. 14.  0. 44. 67.]
 [18.  1.  0. 37. 51.]
 [25. 13.  8. 35. 37.]
 [28. 25.  0. 53. 81.]]


`Sobel`
---

---

In [34]:
import numpy as np

def convolve(image: np.ndarray, kernel: np.ndarray) -> np.ndarray:

    kernel_y, kernel_x = kernel.shape
    image_y, image_x = image.shape
    pad_y, pad_x = kernel_y // 2, kernel_x // 2 # Pad size based on kernel dimensions

    filtered_image = np.zeros_like(a = image, dtype = np.float32)
    paddded_image = np.pad(array = image, pad_width = (( pad_y, pad_x ), ( pad_x, pad_y )), mode = 'constant', constant_values = 0)

    # Convolve
    for y in range(image_y):
        for x in range(image_x):
            # Sliding window
            window = paddded_image[ y : y + kernel_y, x : x + kernel_x ]
            # Apply filters
            filtered_image[ y, x ] = np.sum(window * kernel)

    return filtered_image.astype(dtype = np.float32)


def sobel_filter(image: np.ndarray, axis: str | None = None, div: int | float = 1) -> np.ndarray:
    # Sobel x kernel
    sobel = np.array([[-1, 0, 1],
                       [-2, 0, 2],
                       [-1, 0, 1]], dtype = np.float32)
    
    if axis == 'y':
        sobel = sobel.T

    filtered_image = convolve(image, sobel)
    filtered_image /= div

    return filtered_image


image = np.array([[10, 11, 9, 25, 22],
                   [8, 10, 9, 26, 28],
                   [9, 8, 9, 24, 25],
                   [11, 11, 12, 23, 22],
                   [10, 11, 9, 22, 25]], dtype = np.float32)

print(sobel_filter(image = image, axis = 'y', div = 8))
# print(sobel_filter(image = image, div = 8))

[[  3.25    4.625   6.75   11.125  10.25 ]
 [ -0.625  -0.875  -0.5     0.125   0.625]
 [  0.875   1.      0.5    -1.125  -1.875]
 [  0.625   0.875   0.125  -0.5    -0.25 ]
 [ -4.125  -5.625  -7.25  -10.     -8.375]]


`Gradient Magnitude`
---

---

In [2]:
import numpy as np

def convolve(image: np.ndarray, kernel: np.ndarray) -> np.ndarray:

    kernel_y, kernel_x = kernel.shape
    image_y, image_x = image.shape
    pad_y, pad_x = kernel_y // 2, kernel_x // 2 # Pad size based on kernel dimensions

    filtered_image = np.zeros_like(a = image, dtype = np.float32)
    paddded_image = np.pad(array = image, pad_width = (( pad_y, pad_x ), ( pad_x, pad_y )), mode = 'constant', constant_values = 0)

    # Convolve
    for y in range(image_y):
        for x in range(image_x):
            # Sliding window
            window = paddded_image[ y : y + kernel_y, x : x + kernel_x ]
            # Apply filters
            filtered_image[ y, x ] = np.sum(window * kernel)

    return filtered_image.astype(dtype = np.float32)


def sobel_filter(image: np.ndarray, axis: str | None = None, div: int | float = 1) -> np.ndarray:
    # Sobel kernel
    sobel = np.array([[-1, 0, 1],
                       [-2, 0, 2],
                       [-1, 0, 1]], dtype = np.float32)
    
    if axis == 'y':
        sobel = sobel.T

    filtered_image = convolve(image, sobel)
    filtered_image /= div

    return filtered_image


def gradient_magnitude(image: np.ndarray) -> np.ndarray:

    g_x, g_y = sobel_filter(image, div = 8), sobel_filter(image, axis = 'y', div = 8)
    return (g_magnitude := np.sqrt(g_x**2 + g_y**2))

array = np.array([[11, 15, 20, 21, 22],
                  [8, 19, 17, 22, 23],
                  [7, 14, 18, 21, 25],
                  [10, 15, 15, 19, 54],
                  [11, 12, 11, 17, 22]], dtype = np.float32)


print(gradient_magnitude(array))

[[ 7.5270348  8.567745   9.560661  10.574143  11.672618 ]
 [ 8.450222   4.854122   2.4558604  2.6279745 10.776131 ]
 [ 7.75       4.609772   2.9633174  7.9155703 12.72915  ]
 [ 7.040064   2.766993   3.535534  12.209115   9.581884 ]
 [ 6.5502863  6.903351   8.18917   15.39582   17.201925 ]]
